In [1]:
import pandas as pd
import numpy as np
import pathlib
import os

In [2]:
info_file="C:\\Users\\vamshi krishna\\Desktop\\info.txt"

In [3]:
f=open(info_file)
l = f.read().splitlines()
print(l[0] + "\n" + l[1] + "\n" + l[2] + "\n" + l[3] + "\n" + l[4] + "\n" )

lenfive ratingone Can't change profile picture.
lentwentyfive ratingfour Doesnt seem to be working everytime i go on it, it says it cant update anything so stuck with statuses from november saved on there!
lenfour ratingone Take forever to upload ....
leneight ratingone Painfully slow when it does decide to work.
leneleven ratingone I continually experience freezing, black screens, my phone gets locked up...



In [4]:
f.close()

# Now lets us do preprocessing

First we will extract unique words from "info_file" and add it to the dictonary

In [5]:
import re
def add_words_to_dict(file_name):
    stop_words = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being",
                  "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have",
                  "having", "he", "hed", "hell", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how", "hows", "i", "id", "ill", "im", "ive",
                  "if", "in", "into", "is", "it", "its", "its", "itself", "lets", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
                  "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "shed", "shell", "shes", "should", "so",
                  "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then", "there", "theres", "these", "they", "theyd", "theyll", "theyre",
                  "theyve", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "wed", "well", "were", "weve", "were", "what", "whats", "when",
                  "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why", "whys", "with", "would", "you", "youd", "youll", "youre", "youve",
                  "your", "yours", "yourself", "yourselves", "need", "needed", "can", "u", "every", "rather", "gonna", "m", "tap", "fish", "gives", "nice", "hoping"]
    word_id={}
    
    with open(file_name) as f:
        lines=f.read().splitlines()
    for line in lines:
        line=re.sub(r'[^a-zA-Z\s]', "", line)
        words=line.split()
        words=words[2:]
        for word in words:
            word=word.lower()
            if word not in stop_words:
                word_id[word]=0
                
    id=0
    for word in word_id:
        word_id[word]=id
        id+=1
        
    return word_id

In [6]:
dictonary=add_words_to_dict(info_file)

In [7]:
c=0
for key,value in dictonary.items():
    print(key + " - " + str(value))
    c+=1
    if c==5:
        break

cant - 0
change - 1
profile - 2
picture - 3
doesnt - 4


In [8]:
print("Number of unique words in info_file = " + str(len(dictonary)))

Number of unique words in info_file = 974


Make feautue matrix where first 6 columns represents rating of the app and next 974 columns are the unique words in the info_file, so, a total of 980 features are formed.

Format of each app review is as follows: "lenfour ratingone Notifications are falling apart."

In [9]:
def get_feature_matrix(file_name, word_id):
    rating_bits=6
    cols=rating_bits+len(word_id) 
    data_list=[]
    with open(file_name) as f:
        lines=f.read().splitlines() #Split lines in document.
    for line in lines:
        line=line.lower()
        rating=0
        try:
            rating_str=line.split(" ", 2)[1] #extract rating by the string two times at white space.
        except IndexError:
            continue
        if rating_str.endswith("one"):
            rating=1
        elif rating_str.endswith("two"):
            rating=2
        elif rating_str.endswith("three"):
            rating=3
        elif rating_str.endswith("four"):
            rating=4
        elif rating_str.endswith("five"):
            rating=5
        
        line=line.split(" ", 2)[2] #Read the remaining review excluding length of review and rating.
        line=re.sub(r'[\.\?]', ",", line)
        reviews=line.split(",")
        for review in reviews:
            if review=="":
                continue
            if re.match(r'[^a-zA-Z]', review) is not None:
                continue
            instance=np.zeros((cols, ), dtype=int) #initialize all columns to zero.
            instance[rating]=1
            words=review.split()
            for word in words:
                attr_idx=word_id.get(word, None)
                if attr_idx is not None:
                    instance[rating_bits+attr_idx]=1
                if np.count_nonzero(instance[rating_bits:])!=0: #if all columns are not zero, then append
                    data_list.append(instance)
    
    ret_val=np.array(data_list)
    return ret_val

In [10]:
non_info_file="C:\\Users\\vamshi krishna\\Desktop\\non-info.txt"

# Prepare training dataset

In [11]:
training_data_info = get_feature_matrix(info_file, dictonary)
training_data_non_info = get_feature_matrix(non_info_file, dictonary)

In [12]:
training_y_info_label = np.ones(training_data_info.shape[0], dtype=int)
training_y_label = np.append(training_y_info_label, np.zeros(training_data_non_info.shape[0], dtype=int))

In [13]:
training_data = np.append(training_data_info, training_data_non_info, axis=0)

So, we got training data with X-feature matrix (training_data) and y-matrix (training_y_label)

Lets us import test data set and will repeat the same process

In [14]:
test_info="C:\\Users\\vamshi krishna\\Desktop\\test\\info.txt"
test_non_info="C:\\Users\\vamshi krishna\\Desktop\\test\\non-info.txt"

In [15]:
test_x0 = get_feature_matrix(test_info, dictonary)
test_y = np.ones(test_x0.shape[0], dtype=int)
test_x1 = get_feature_matrix(test_non_info, dictonary)
test_y = np.append(test_y, np.zeros(test_x1.shape[0], dtype=int))
test_x = np.append(test_x0, test_x1, axis=0)

Let's perform the powerful machine learning algorithm named Naive Bayes :)

In [16]:
from sklearn.naive_bayes import BernoulliNB
clf=BernoulliNB()
clf.fit(training_data, training_y_label)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [17]:
clf.score(test_x, test_y)

0.8238218333664744

In [18]:
result=clf.predict(test_x)
result=np.array(result, dtype=bool)

Lets try our model on unlabeled dataset and we divide that dataset into informative and non_informative.

In [19]:
unlabeled_data="C:\\Users\\vamshi krishna\\Desktop\\trainU\\New Text Document.txt"

In [20]:
unlabeled_matrix = get_feature_matrix(unlabeled_data, dictonary)

In [21]:
final_result = clf.predict(unlabeled_matrix)

In [22]:
final_result = np.array(final_result, dtype=bool)

In [23]:
ones_count = np.count_nonzero(final_result)

In [24]:
final_result.shape

(19815,)

In [25]:
percent = ones_count/19815

In [26]:
percent

0.6283118849356548

In [27]:
print("Precentage of informative reviews in unlabeled dataset is : " + str(percent*100))

Precentage of informative reviews in unlabeled dataset is : 62.831188493565485


# Lets put these these informative reviews in new document

In [28]:
f = open("C:\\Users\\vamshi krishna\\Desktop\\trainU\\New Text Document.txt", 'r+')
Lines = f.readlines() 

In [29]:
new_file=open("C:\\Users\\vamshi krishna\\Desktop\\infor_file_new.txt", 'a+') # infor_file_new is an empty text document

In [44]:
i=0;
for line in Lines:
    if(final_result[i]==1):
        new_file.write(line)
    i+=1

# The file named "infor_file_new" contains informative reviews of the unlabelled dataset.

# That completes mining the app reviews